In [158]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import sqlite3 as db
import datetime as dt
from sklearn.metrics import mean_squared_error

In [159]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [160]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [161]:
#baseline request from the application layer
#outputs a list of birds at the stop and a classification based solely off the number of observations
def birdList_request(StopName: str,cnx):
    try:
        query = f'SELECT speciesCode,count(subId) as "checklists",(SELECT count(subId) FROM historicObservations hxobx WHERE hxobx.speciesCode=hsob.speciesCode) as "sightings" FROM historicObservations hsob LEFT JOIN closestStop on hsob.locId=closestStop.locId WHERE StopName = "{StopName}" GROUP BY speciesCode;'
        sightings = pd.read_sql(sql=query,con=cnx)
    #rareness at the stop
        sightings['stopGroup'] = int
        bucket = sightings['checklists'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['checklists'] <= bucket[0.15],'stopGroup'] = 1  #mythic
        sightings.loc[(sightings['checklists'] > bucket[0.15]) & (sightings['checklists'] <= bucket[0.5]),'stopGroup'] = 2  #rare
        sightings.loc[(sightings['checklists'] > bucket[0.5]) & (sightings['checklists'] < bucket[0.85]),'stopGroup'] = 3   #uncommon
        sightings.loc[(sightings['checklists'] >= bucket[0.85]) & (sightings['checklists'] <=bucket[1]),'stopGroup'] = 4    #common
    #overall rareness
        sightings['overall'] = int
        bucket = sightings['sightings'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['sightings'] <= bucket[0.15],'overall'] = 1
        sightings.loc[(sightings['sightings'] > bucket[0.15]) & (sightings['sightings'] <= bucket[0.5]),'overall'] = 2
        sightings.loc[(sightings['sightings'] > bucket[0.5]) & (sightings['sightings'] < bucket[0.85]),'overall'] = 3
        sightings.loc[(sightings['sightings'] >= bucket[0.85]) & (sightings['sightings'] <=bucket[1]),'overall'] = 4
        sightings['StopName'] = StopName
    #raise an exception if the stopName given is not valid and return a list of valid stop names
    except Exception as ex:
        raise ex
    return sightings

LSTM analysis of common birds

In [162]:
commonCore = birdList_request(StopName='EastSunsetWay',cnx=connectDB())
commonCore = commonCore[commonCore.apply(lambda x: (x['stopGroup']==4) and (x['overall']==4),axis=1)]

In [227]:
querySpecies = 'amerob'
StopName = 'EastSunsetWay'
query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.speciesCode = "{querySpecies}" AND StopName = "{StopName}";'
obsData = pd.read_sql(query,con=connectDB(),parse_dates=['obsDt'])
obsData.set_index('obsDt',inplace=True)

In [228]:
obsDataDaily = obsData.resample('d')
dailyData = obsDataDaily.mean()
dailyData

,howMany
obsDt,
2019-03-23,5.0
2019-03-24,NaN
2019-03-25,NaN
2019-03-26,NaN
2019-03-27,NaN
...,...
2022-03-06,NaN
2022-03-07,NaN
2022-03-08,NaN


In [165]:
##TODO update to bidirectional model
##TODO update LSTM from days of the week to weeks of the year
##TODO build realtime prediction function using LSTM model

In [166]:
##TODO upgrade the interpolation method for dailyData in the LSTM module
fdd = dailyData['howMany'].interpolate(method='slinear',limit_direction='both')
fdd = pd.DataFrame(fdd)

In [229]:
#dailyData.reset_index(inplace=True)
dailyData['mask'] = dailyData['howMany'].interpolate(method='zero',limit_direction='both',limit=21)
dailyData.loc[(dailyData['mask'].isna() == True),'howMany'] = 0
testDailyData = dailyData[dailyData['howMany'].notna()].drop(columns='mask')
testDailyData

,howMany
obsDt,
2019-03-23,5.0
2019-03-28,3.0
2019-03-31,1.0
2019-04-03,47.0
2019-04-24,4.0
...,...
2022-01-05,1.0
2022-01-21,3.0
2022-02-02,8.0


In [203]:
from sklearn.metrics import r2_score

In [232]:
testResults = []
methodDict = [{'method':'linear','order':3,'limit_direction':'both','limit':21},{'method':'slinear','order':3,'limit_direction':'both','limit':21},{'method':'quadratic','order':0,'limit_direction':'both','limit':21},{'method':'cubic','order':3,'limit_direction':'both','limit':21},{'method':'spline','order':3,'limit_direction':'both','limit':21},{'method':'spline','order':5,'limit_direction':'both','limit':21},{'method':'polynomial','order':3,'limit_direction':'both','limit':21},{'method':'polynomial','order':5,'limit_direction':'both','limit':21}]
for v in list([0.2,0.25,0.3,0.35]):
    testDailyData['sample'] = testDailyData['howMany'].sample(frac=v,random_state=1)
    blob = {}
    for test in methodDict:
        testName = str(test['method'])
        testDailyData[testName] = testDailyData['sample']
        testDailyData[testName] = testDailyData['sample'].interpolate(method=test['method'],order=test['order'],limit=test['limit'],limit_direction=test['limit_direction']).fillna(testDailyData['howMany'])
        r_sqr = r2_score(y_true=testDailyData['howMany'],y_pred=testDailyData[testName])
        if r_sqr < 0:
            continue
        blob = {'method':test['method'],'order':test['order'],'limit':test['limit'],'sampleSize':v,'r_sqr':r_sqr}
        testResults.append(blob)
if not testResults:
    testResults.append({'method':'nearest','order':0,'limit':2})
    resultFrame = pd.DataFrame.from_dict(testResults)
else:
    resultFrame = pd.DataFrame.from_dict(testResults)
    resultFrame = resultFrame.groupby(by=['method','order','limit'])['r_sqr'].median().reset_index()
    resultFrame.sort_values(by=['r_sqr'],ascending=False,inplace=True,ignore_index=True)

In [233]:
resultFrame

,method,order,limit,r_sqr
0,slinear,3,21,0.626317
1,quadratic,0,21,0.495720
2,cubic,3,21,0.474433
3,polynomial,3,21,0.474433
4,linear,3,21,0.214105
5,spline,3,21,0.183916


In [237]:
#dailyData = dailyData.drop(columns='mask')
#dailyData['howMany'] = dailyData['howMany'].interpolate(method='slinear',limit=21,limit_direction='both',order=3)
dailyData = fdd
fdd

,howMany
obsDt,
2019-03-23,5.000000
2019-03-24,4.600000
2019-03-25,4.200000
2019-03-26,3.800000
2019-03-27,3.400000
...,...
2022-03-06,12.714286
2022-03-07,12.785714
2022-03-08,12.857143


In [238]:
def split_trainTest(dailyObs):
    train, test = dailyObs[1:-327],dailyObs[-328:-6]
    train = np.array(np.split(train,len(train)/7))
    test = np.array(np.split(test,len(test)/7))
    return train, test

In [239]:
train, test = split_trainTest(dailyObs=fdd)
print(train.shape)
print(test.shape)

(108, 7, 1)
(46, 7, 1)


In [240]:
#build forward-moving windows
def to_supervised(train,nInput,nOut = 7):
    train = train.reshape((train.shape[0]*train.shape[1],train.shape[2]))
    X, y = [],[]
    inStart = 0
    for _ in range(len(train)):
        inEnd = inStart + nInput
        outEnd = inEnd + nOut
        if outEnd <= len(train):
            xIn = train[inStart:inEnd,0]
            xIn = xIn.reshape((len(xIn),1))
            X.append(xIn)
            y.append(train[inEnd:outEnd,0])
        inStart+= 1
    return np.array(X),np.array(y)

In [241]:
X, y = to_supervised(train=train,nInput=7)
print(X.shape)
print(y.shape)

(743, 7, 1)
(743, 7)


In [242]:
from keras.layers import Bidirectional

In [243]:
def buildModel(train,nInput):
    train_x,train_y = to_supervised(train,nInput)
    verbose, epochs, batch_size = 0,70,16
    nTimesteps,nFeatures,nOutputs = train_x.shape[1],train_x.shape[2],train_y.shape[1]
    model = Sequential()
    #model.add(Bidirectional(LSTM(units=200,activation='relu')))
    model.add(LSTM(units=200,activation='relu', input_shape=(nTimesteps,nFeatures)))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(nOutputs))
    model.compile(loss='mse',optimizer='adam')
    model.fit(train_x,train_y,epochs=epochs,batch_size=batch_size,verbose=verbose)
    return model

In [ ]:
##TODO: build another model using encoder-decoder, run tests

In [244]:
def forecast(model,history,nInput):
    histData = np.array(history)
    #histData = np.array(np.split(histData,len(histData)/7))
    histData = histData.reshape((histData.shape[0]*histData.shape[1],histData.shape[2]))
    inputX = histData[-nInput:,0]
    inputX = inputX.reshape((1,len(inputX),1))
    yhat = model.predict(inputX,verbose=0)
    yhat = yhat[0]
    return yhat

In [245]:
def eval_forecasts(actual,predicted):
    try:
        scores = list()
        for i in range(actual.shape[1]):
            mse = mean_squared_error(actual[:,i],predicted[:,i])
            rmse = np.sqrt(mse)
            scores.append(rmse)
        s = 0
        for row in range(actual.shape[0]):
            for col in range(actual.shape[1]):
                s+= (actual[row,col] - predicted[row,col])**2
        score = np.sqrt(s/(actual.shape[0]*actual.shape[1]))
    except Exception as metricExc:
        raise metricExc
    return score, scores

In [246]:
def evalModel(train,test,nInput):
    model = buildModel(train=train,nInput=nInput)
    history = [x for x in train]
    predictions = []
    for i in range(len(test)):
        yhatSeq = forecast(model=model,history=history,nInput=nInput)
        predictions.append(yhatSeq)
        history.append(test[i,:])
    predictions = np.array(predictions)
    score, scores = eval_forecasts(actual=test[:,:,0],predicted=predictions)
    return score, scores
    #return predictions

In [255]:
score, scores = evalModel(train=train,test=test,nInput=7)


In [256]:
def summarizeEvalScore(name,score,scores):
    sScores = ', '.join(['%f' % s for s in scores])
    print('%s: [%.3f] %s' % (name, score, sScores))

In [257]:
summarizeEvalScore(name='run',score=score,scores=scores)

run: [3.388] 3.699561, 3.183253, 3.239663, 3.066359, 3.069039, 3.652617, 3.725892


1. run1: [2.551] 2.458957, 2.472114, 2.475770, 2.677192, 3.045214, 2.251491, 2.402137 n = 7
2. run2: [2.655] 2.482476, 2.564362, 2.557338, 2.738337, 3.200391, 2.457562, 2.509320 n = 21
3. run3: [3.221] 2.561701, 2.617324, 2.715710, 3.185073, 3.910806, 3.548933, 3.717976 n = 14
4. run4: [2.757] 2.647477, 2.664606, 2.711631, 2.933634, 3.188165, 2.496027, 2.599233 n = 7 bidirectional
5. run5: [2.216] 0.385453, 0.939606, 1.419237, 1.818103, 2.423542, 2.977693, 3.646165 n = 7 note: changed interpolation method to slinear
6. run6: [2.442] 0.468001, 1.068273, 1.803750, 2.283598, 2.770545, 3.216125, 3.727616 n = 7
7. run7: [2.146] 0.346224, 1.057628, 1.474756, 1.910932, 2.273262, 2.820911, 3.470625 n = 7
8. run: [2.082] 0.474444, 0.973971, 1.394103, 1.790021, 2.196929, 2.821583, 3.353662 n = 7 improved interpolation method
9. run: [2.770] 1.373590, 0.953020, 1.333994, 2.028866, 2.949500, 3.854387, 4.632758 n = 28 stick with a lower n